In [1]:
import argparse
from w2vEmbReader import W2VEmbReader as EmbReader
from keras.layers import Input, Embedding
from keras.models import Model
from keras.constraints import MaxNorm
import numpy as np
import keras.backend as K
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from tqdm import tqdm
from sklearn.metrics import classification_report,precision_score,confusion_matrix
import keras.backend as K
from keras.preprocessing import sequence

import utils as U
import reader as dataset

Using Theano backend.
2020-10-10 15:58:27,542 WARNING g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda\envs\keras_theano\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
2020-10-10 15:58:27,545 WARNING g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
2020-10-10 15:58:27,847 WARNING Using NumPy C-API based implementation for BLAS functions.


In [2]:
#parser=argparse.ArgumentParser()
parser = U.add_common_args()
parser.add_argument("-as", "--aspect_size", dest="aspect_size", type=int, metavar='<int>', default=14,
                    help="The number of aspects specified by users (default=14)")
parser.add_argument("-asrange", "--aspect_range", nargs='+', dest="aspect_range", type=int, metavar='<int>', default=14,help="Range of aspects specified by users (default=14)")
parser.add_argument("--emb-name",  type=str, help="The name to the word embeddings file", default="w2v_64k_unigram_100d.model")
#parser.add_argument("-o", "--out-dir", dest="out_dir_path", type=str, metavar='<str>',
#                        help="The path to the output directory", default="output")
args=parser.parse_args()

#out_dir = args.out_dir_path

vocab, train_x, test_x, overall_maxlen = dataset.get_data(args.domain, vocab_size=args.vocab_size, maxlen=args.maxlen)

import pickle
with open('vocab_c.pkl','rb') as f:
    vocab=pickle.load(f)


emb_reader = EmbReader(os.path.join("..", "preprocessed_data"), args.emb_name)



sym = Input(shape=(None,), dtype='int32', name='sym')
word_emb_l = Embedding(len(vocab), 100, mask_zero=True, name='word_emb_l', embeddings_constraint=MaxNorm(10))
e_sym = word_emb_l(sym)
mod = Model(inputs=[sym], outputs=[e_sym])
embs = mod.get_layer('word_emb_l').embeddings
K.set_value(embs, emb_reader.get_emb_matrix_given_vocab(vocab, K.get_value(embs)))
word_emb = K.get_value(mod.get_layer('word_emb_l').embeddings)
word_emb = word_emb / np.linalg.norm(word_emb, axis=-1, keepdims=True)

sent_emb=[]
sent_avg_emb=[]
for sentence in test_x:
    for word in sentence:
        sent_emb.append(emb_reader.embeddings[word])
    sent_avg_emb.append(sum(sent_emb)/len(sent_emb))


with open('sent_emb.txt','w') as f:
    f.write(str(sent_avg_emb))

with open('sent_emb.pkl','wb') as f:
    pickle.dump(f,sent_avg_emb)

usage: ipykernel_launcher.py [-h] [-o <str>] [-b <int>] [-v <int>]
                             [--maxlen <int>] [--domain <str>] [-as <int>]
                             [-asrange <int> [<int> ...]]
                             [--emb-name EMB_NAME]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Sudeshna Dasgupta\AppData\Roaming\jupyter\runtime\kernel-15bbd563-3445-4626-b815-eff5007b8be6.json


SystemExit: 2

C:\Anaconda\envs\keras_theano\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 2

In [3]:
from w2vEmbReader import W2VEmbReader as EmbReader
embname='../preprocessed_data/sentihood/glove.6B.100d.txt.word2vec'
emb_reader = EmbReader(os.path.join("..", "preprocessed_data"), embname)

2020-10-10 15:58:38,917 INFO Loading embeddings from: ..\preprocessed_data\../preprocessed_data/sentihood/glove.6B.100d.txt.word2vec


Loading embeddings from: ..\preprocessed_data\../preprocessed_data/sentihood/glove.6B.100d.txt.word2vec


2020-10-10 15:58:38,918 INFO loading projection weights from ..\preprocessed_data\../preprocessed_data/sentihood/glove.6B.100d.txt.word2vec
2020-10-10 15:59:17,554 INFO loaded (400000, 100) matrix from ..\preprocessed_data\../preprocessed_data/sentihood/glove.6B.100d.txt.word2vec
2020-10-10 16:00:38,270 INFO   #vectors: 400000, #dimensions: 100


In [5]:
sentence=[3, 138, 5, 37, 8]
sent_emb=[]
for word in sentence:
    sent_emb.append(emb_reader.embeddings[word])
print(sent_emb)

KeyError: 3

In [6]:
print(type(emb_reader.embeddings)) #dictionary of length 40000 , pretrained glove 100d

<class 'dict'>


In [7]:
with open(r'C:\Users\Sudeshna Dasgupta\Documents\Thesis_GuidedResearch\thesis\Aspect Extraction\Baseline models\preprocessed_data\sentihood\test.txt','r') as f:
   test=f.readlines()
#emb_reader.embeddings[]

In [8]:
test_wordlist=[i.split() for i in test]

In [9]:
sent_avg_emb=[]
for sentence in test_wordlist:
    sent_emb=[]
    for word in sentence:
        try:
            sent_emb.append(emb_reader.embeddings[word])
        except KeyError:
           sent_emb.append([0]*100) #emb dim 100
    print(sent_emb)
    sent_emb_matrix=np.array(sent_emb)
    sent_avg_emb.append(np.mean(sent_emb_matrix, axis=0))
    #sent_avg_emb.append(sum(sent_emb)/len(sentence))


with open('sent_avg_emb.txt','w') as f:
    f.write(str(sent_avg_emb))



[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-0.14471, -0.16199, 0.32201, 0.79056, 0.41845, -0.42938, -1.1759, -0.29589, -0.29759, 0.31202, -0.90897, -0.26276, -0.09649, -0.026413, 0.26833, -0.76579, -0.39476, 0.29882, -0.71515, 0.42106, 0.061787, 0.10195, -0.75105, -0.54249, -0.17525, 0.13127, 0.29025, -0.27829, -0.064917, -0.095284, 0.25912, 0.22806, -0.13962, 0.013549, 0.47956, -0.42568, 0.92421, 0.6654, -0.63769, 0.46098, -0.28924, -0.82379, -0.024738, -0.7458, 0.026292, -0.15056, 1.3731, -0.48979, -0.19293, -0.28239, -0.20675, 0.12548, -0.57321, 0.88322, 0.37409, -2.1248, 0.33869, -1.2603, 1.4491, -0.41792, 0.068244, 0.31072, -1.1232, -0.0307, 0.74356, 0.15355, -0.47774, -0.35084, 0.92321, 0.21293, 0.25014, -0.36026, -0.34419,

[[0.70065, -0.15386, -0.22752, -0.28593, -0.10264, 1.0615, -0.16303, 0.69502, 0.82531, 1.3548, -0.63637, -0.98747, 0.27273, -0.48764, 0.27201, 0.15802, -0.13128, -0.26218, -0.59824, 0.43229, 0.42291, 0.35024, -0.16275, -0.95404, -0.39683, -0.56042, -0.23703, -0.923, -0.5658, 0.49793, 0.055919, -0.25884, 1.114, 0.87816, 0.38069, -0.018469, 0.47632, 0.01487, 0.98658, 0.28896, 0.17007, -0.6762, -0.0044398, -0.35035, 0.32092, -0.17451, -0.08832, 1.0067, 0.42836, -0.075427, -0.92768, -0.43551, -0.21658, 0.3306, -0.47306, -2.0216, -0.3941, -0.29347, 1.2939, 0.16341, -0.34345, 0.90405, 0.53012, -0.7037, 0.65555, -0.17659, 0.23691, -0.40494, 0.034622, -0.075822, -0.36552, -0.24991, 0.031973, -0.98546, 0.29763, -0.46197, 0.53207, 0.30199, -0.60769, -0.048772, -0.14528, 0.84266, -0.54366, 0.1307, -0.16423, 0.20399, -0.91924, -0.53841, -0.011432, -0.059142, 0.43665, -0.17238, -0.0405, 0.054936, -1.2149, 0.66897, 0.32061, 0.37405, 0.58181, 0.80017], [0.049432, 0.29907, 0.070318, 0.085277, -0.17125

[[0.0024119, 0.50144, 0.16453, -0.23827, -1.3334, 0.21536, -0.22754, 0.88037, 0.96489, -0.84845, 0.04929, 0.6638, 0.72793, -0.59508, -0.061451, 0.42035, 0.2608, 0.75803, -0.37254, 0.368, 0.29604, 0.6933, -0.38568, -0.91991, 0.20945, 0.035935, 0.060378, -0.072355, 0.80588, -1.3924, -0.51972, 0.023169, 0.21641, 0.23588, -0.17542, 0.54578, 0.097823, 0.47507, 0.35519, 0.1715, 0.29964, 0.75809, -0.59485, -0.1754, -0.89226, -0.34347, -0.27042, -0.33202, -0.37249, 0.002517, -0.018512, 0.81792, -0.95083, -0.42594, -0.12798, -0.048849, 0.76817, 0.35467, -0.87669, -0.23881, -0.30963, 0.28486, -0.97344, -0.070903, 0.078204, 0.67811, 0.0060645, -0.17977, -0.8888, -0.064554, 0.35943, 0.029266, 0.26726, -0.91049, -0.13037, -0.51428, 0.36728, -0.22366, 0.11019, -0.92972, 0.49366, -0.58004, -0.97384, 0.028854, -0.3722, -0.33347, 0.28816, -0.486, -0.24828, -0.18961, 0.054812, -0.25718, 0.070229, -0.49131, -0.31793, 0.091763, 0.016464, -0.63132, -0.37671, 0.38152], [-0.3333, -0.14814, -0.027924, -0.1579

[[-0.037766, 0.53881, 0.75407, -0.18197, -0.64225, 0.85587, -0.6885, 0.30163, 0.27817, -1.1247, 0.1609, 0.21545, -0.12533, 0.067352, 0.055924, -0.31621, -0.69537, 0.41979, -0.46417, 0.59416, -0.14298, -0.51765, 0.074196, -0.87629, -0.076729, -0.066771, -0.041785, 0.052378, 1.4092, 0.15677, 0.10753, -0.0083313, 0.30327, 0.67538, -0.55822, 0.28002, -0.18447, -0.3375, 1.0683, -0.58895, -0.48451, -0.006247, -0.26566, -0.83246, -0.49781, -0.28771, 0.12445, -0.69842, 0.030756, -1.4498, 0.44712, -0.22895, 0.4209, 0.11722, -0.066422, -1.4403, 0.10916, 0.093753, 1.3541, 0.15767, -0.02006, 1.1728, -0.22594, -0.61175, 0.4487, -0.070912, 0.65395, 0.90912, -0.93871, -0.14871, 0.27689, -0.14524, 0.1947, -1.0849, 0.065284, 0.5539, 0.064031, -0.71823, 0.11073, -0.56209, 0.37111, -0.82066, -0.19952, -0.17456, -1.6247, -0.38816, -0.022604, -0.028887, -0.078509, 0.049833, 0.77831, -0.33463, -0.66798, -0.19821, -0.09095, 0.23492, -0.68558, 0.18655, 0.46215, 0.37699], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

[[-0.19981, -0.21476, -0.32423, -0.86034, -0.25049, -0.59644, 0.1415, 0.57725, 0.43117, -0.31747, -0.09928, 0.26806, 0.1554, 0.3317, -0.15265, 0.23805, 0.20762, -0.32435, -0.072702, 0.86175, 0.094382, 0.39243, -0.24238, 1.1357, 0.55373, 0.087505, -0.35756, -0.19667, 0.45664, -0.72949, -0.48245, 0.12889, 0.26522, 0.073711, -0.34625, -0.26454, 0.38874, 0.82623, -0.26811, 0.13476, 0.57384, 0.91631, -0.72547, 0.21586, -0.50148, 0.17393, -0.88855, 0.75923, 0.52758, 0.49115, 0.33361, -0.020822, 0.35818, 0.31121, -0.38758, 0.031135, 0.21907, 0.34891, -1.3017, -0.4685, -0.11966, -0.13973, -0.0073455, 0.89201, 0.14101, 1.0691, 1.3284, 0.84679, -0.33094, 0.47885, 0.5661, -0.39228, 0.36043, 0.48183, 0.44945, 0.7024, -0.22754, -0.79365, -0.080359, -0.98462, -0.59217, -0.076494, 0.66632, 1.173, -0.19215, -0.23712, 0.066745, -0.42115, 0.33924, -0.43616, -0.19605, -0.035156, -0.65427, 0.12185, -0.019766, -0.023336, 0.39518, 0.28896, 0.27131, -0.12734], [0.49798, -0.19195, -0.042257, 0.30716, 0.14201,

...

[[-0.22874, 0.36872, 0.31896, -0.74239, 0.027424, 0.79898, -0.082443, 0.74086, -0.029152, -0.22086, -0.25652, 0.24452, 0.52154, -0.00027352, -0.088076, -0.54444, 0.38625, 0.063671, -0.27383, 0.7522, 0.54208, -0.16792, 0.86998, 0.21324, 0.64398, -0.35221, 0.40224, 0.1457, 0.8393, 0.74675, -0.27532, 0.60076, 0.09879, 0.88763, 0.94413, 0.33671, 0.39813, 0.19962, -0.023724, -0.89748, -0.25989, 0.32126, -0.61065, 0.36345, 0.071808, -0.42589, -0.57628, -0.58397, -0.1649, -1.0589, -0.52296, -0.33326, 0.62457, 0.39041, 0.14096, -2.5526, 0.0057586, -0.63765, 1.181, 0.075413, 0.07262, 1.4135, -0.27231, -1.3855, 1.0909, 0.43547, 0.51665, -0.51073, 0.16852, -0.32413, -0.13939, -0.29631, 0.23774, -0.0086583, 0.027088, 0.24738, -0.29137, -0.16888, 0.15726, -0.12054, 0.56824, -0.031604, 0.16588, 0.6613, -1.2611, 0.44062, 0.1888, -0.043579, -0.3021, 0.55952, 0.22522, -0.44779, 0.47641, -0.047689, -0.91124, -0.46053, -0.4224, 0.28768, 1.1311, 0.063564], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[[-0.57548, -0.043236, -0.1972, 0.58541, -0.57189, 0.16221, -0.70626, 0.53485, -0.31428, -0.13641, -0.12833, 0.010291, -0.20692, -0.557, -0.17515, -0.082819, 0.12148, -0.1396, -0.57657, 0.47053, -0.69398, -0.56833, -0.78928, -0.27884, -0.64573, 0.5565, 0.060804, -0.64606, -0.25518, 0.31791, 0.12194, 0.048183, -0.64601, -0.38327, 0.12061, 0.74751, 0.10604, -0.52138, -0.017322, -0.32586, -0.23957, 0.29111, 0.015217, -1.2517, -0.33418, 0.23553, -0.203, -0.67543, -0.92901, -1.3482, 0.65388, -0.35113, -0.56663, 0.17247, -0.78661, -0.61462, 0.63937, -0.38254, 1.1183, 0.18329, -0.25046, 0.17942, 0.49362, 0.41061, 0.65019, 0.14131, -0.056467, 1.0308, -0.15851, -0.5061, 0.18965, -0.24445, -0.29897, -0.086214, -0.72705, -0.19503, -0.60543, -0.048642, -0.048641, -0.87168, 0.021548, -0.68802, -0.56679, -0.23595, -0.96225, -0.32152, 0.76583, 0.00037482, -0.75852, -0.17146, 0.18943, -0.2736, -0.014518, -0.73438, -0.18552, 0.21676, 0.13206, -0.10507, 0.26555, 0.32193], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[[-0.41616, -0.26538, 0.21721, -0.26015, -0.18044, 0.38745, -0.28837, 0.66287, -0.59621, -0.20226, 0.2043, 0.40292, 0.48074, 0.10256, -0.26353, -0.74113, -0.01182, 0.12373, -0.44447, 0.27058, 0.30294, 0.58355, 0.083755, 0.095948, -0.34676, -0.25971, -0.35108, -1.0168, 0.42781, 0.27343, -0.56433, 0.18293, 0.11292, 0.2184, 0.32203, 1.1937, 0.3209, -0.05548, 0.12048, -0.58012, -0.3134, -0.19527, 0.35184, -0.71589, 0.29069, -0.0266, -0.17616, 0.28962, 0.54047, -0.96431, 0.30181, -0.81813, -0.017907, 0.83698, 0.48478, -2.2096, 0.19078, -0.61127, 1.6743, 0.27376, -0.15897, 0.27199, -0.456, -0.55974, 0.4887, 0.73636, -0.04447, 0.1923, -0.80237, -0.39617, 0.12095, -0.54372, -0.19119, -0.56854, 0.11429, 0.18902, -0.42513, -0.17852, -0.056843, -0.32798, 0.29106, -0.019578, -0.69102, -0.05396, -1.03, -0.090328, -0.038899, 0.75633, -0.12612, -0.32224, -0.22403, -0.044219, 0.6454, -0.47888, -0.63891, -0.028902, 0.33386, 0.40199, -0.033789, -0.031944], [-0.10379, -0.014792, 0.59933, -0.51316, -0.036

[[0.35541, -0.21086, -0.057342, -0.39201, -0.85414, 0.336, -0.0037677, -0.26208, -0.17445, 0.46049, -1.1371, 0.52377, 0.12431, -0.25036, 0.83204, 0.50718, -0.8096, -0.4681, 0.39024, -0.21373, -0.14374, -0.24723, 0.13851, -0.055735, 0.021371, -0.46638, 0.89644, -0.96249, -0.48842, -0.40148, 0.72624, 0.5566, -0.63578, 0.23623, -0.0048154, -0.36566, 0.48771, 0.034867, -0.052087, 0.11084, -1.244, 0.22789, -0.68617, -0.20182, -0.69488, -1.1077, 0.58684, -0.51373, -0.82515, -0.69358, 0.64049, 0.16127, -0.79419, 1.0594, -0.26544, -0.87779, 0.29736, -0.37579, 0.94934, -0.51267, -0.084771, -0.3379, 0.073939, 0.32903, 0.86489, -0.20677, 0.45936, -0.5685, -0.37795, 0.493, 0.014915, 0.065963, 0.088365, -0.072337, 0.20721, -0.064105, 0.57898, 0.55605, -0.068127, -0.010939, -0.25761, -0.10052, -0.065281, 0.15322, -0.69545, -0.92752, -1.0085, 0.19392, 0.20221, -0.56222, -1.2128, -0.29775, -0.54096, -0.2645, 1.0424, 0.064485, 0.2404, -0.79614, 0.92753, 0.48381], [-0.22874, 0.36872, 0.31896, -0.74239, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
import pickle

with open('sent_avg_emb.pkl','wb') as f:
    pickle.dump(sent_avg_emb,f)

In [11]:
len(sent_avg_emb[0]) #sentence vectors for test dataset

100

In [12]:
from Aspect_parse import aspect_parse

In [13]:
aspect_dict=aspect_parse(20)

In [19]:
aspect_dict

{}

In [15]:
cluster_emb_matrix=[]
for k,v in aspect_dict.items():
    cluster_emb=[]
    for word in v:
        try:
            #sims = word_emb.dot(desc.T)
            cluster_emb.append(emb_reader.embeddings[word])
        except KeyError:
           cluster_emb.append([0]*100) #emb dim 100
    print(cluster_emb)
    cluster_emb_matrix.append(cluster_emb)
    #cluster_emb_matrix=np.array(cluster_emb)
prediction_labels=[]
for sent_emb in sent_avg_emb:
    cosin=[]
    for cluster in cluster_emb_matrix:
        sims=[]
        for word in cluster:
            sims.append(sent_emb.dot(word))
        cosin.append(sum(sims)/len(sims))
    prediction_labels.append(cosin.index(max(cosin)))


ValueError: max() arg is an empty sequence

In [16]:
np.array(cluster_emb_matrix).shape

(0,)

In [59]:
x=sent_avg_emb[4]
c=cluster_emb_matrix[0]

In [17]:
all_sims=[]
for cluster in cluster_emb_matrix:
    sims=[]
    for word in cluster:
        s=x.dot(word)
        sims.append(s)
    all_sims.append(sum(sims)/len(sims))

In [18]:
all_sims
#5.826008660220953 when x=sent_avg_emb[3]

[]

In [56]:
len(sims), len(all_sims)

(100, 20)

In [57]:
all_sims.index(max(all_sims))

17

In [66]:
prediction

<function __main__.prediction(test_labels, aspect_probs, cluster_map, domain)>

In [69]:
from sklearn.metrics import classification_report,precision_score,confusion_matrix


In [70]:
def evaluation(true, predict, domain):
    true_label = []
    predict_label = []

    if domain == 'restaurant':

        for line in predict:
            predict_label.append(line.strip())

        for line in true:
            true_label.append(line.strip())

        print(classification_report(true_label, predict_label,
                                    ['Food', 'Staff', 'Ambience', 'Anecdotes', 'Price', 'Miscellaneous'], digits=3))

    elif domain == 'drugs_cadec':
        for line in predict:
            predict_label.append(line.strip())

        for line in true:
            true_label.append(line.strip())

        print(classification_report(true_label, predict_label, digits=3))

    else:
        for line in predict:
            label = line.strip()
            if label == 'smell' or label == 'taste':
                label = 'taste+smell'
            predict_label.append(label)

        for line in true:
            label = line.strip()
            if label == 'smell' or label == 'taste':
                label = 'taste+smell'
            true_label.append(label)

        #print(classification_report(true_label, predict_label,
        #                            ['feel', 'taste+smell', 'look', 'overall', 'None'], digits=3))
        print(classification_report(true_label, predict_label, digits=3))
        #cls_dict=classification_report(true_label, predict_label, digits=3,output_dict=True)
        #print(pd.DataFrame.from_dict(cls_dict))
        with open('classification_report.txt','w') as r:
            r.write(classification_report(true_label, predict_label, digits=3))

        #labels to label the confusion matrix
        labels=['live', 'safety', 'price', 'quiet', 'dining', 'nightlife', 'transit-location', 'touristy', 'shopping', 'green-culture', 'multicultural', 'general']
        cm=confusion_matrix(true_label, predict_label,labels=labels)
        print(pd.DataFrame(cm, index=labels, columns=labels))
        pd.DataFrame(cm, index=labels, columns=labels).to_csv('Confusion_Matrix.txt')

        #print micro f1 score
        from sklearn.metrics import f1_score
        print('Micro F1 score\n',f1_score(true_label, predict_label, average='micro'))
        #print('Weighted macro F1 score\n',f1_score(true_label, predict_label, average='weighted'))


def prediction(test_labels, aspect_probs, cluster_map, domain):
    label_ids = np.argsort(aspect_probs, axis=1)[:, -1]
    predict_labels = [cluster_map[label_id] for label_id in label_ids]
    evaluation(open(test_labels), predict_labels, domain)

In [71]:
cluster_map={0: 'general', 1: 'general', 2: 'general', 3: 'general', 4: 'general', 5: 'transit-location',
6: 'transit-location', 7: 'transit-location', 8: 'green-culture', 9: 'price',
10: 'touristy', 11: 'multicultural', 12: 'price', 13: 'general', 14: 'price', 15: 'shopping', 
16: 'general', 17: 'price', 18: 'general', 19: 'general'}
predict_labels = [cluster_map[label_id] for label_id in prediction_labels]
evaluation(open(test_labels), predict_labels, 'Sentihood')

C:\Anaconda\envs\keras_theano\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

          dining      0.000     0.000     0.000        16
         general      0.000     0.000     0.000       346
    green-nature      0.000     0.000     0.000        28
            live      0.000     0.000     0.000        67
   multicultural      0.000     0.000     0.000        37
       nightlife      0.000     0.000     0.000        41
           price      0.144     0.993     0.252       145
           quiet      0.000     0.000     0.000        19
          safety      0.000     0.000     0.000       109
        shopping      0.000     0.000     0.000        43
        touristy      0.000     0.000     0.000        21
transit-location      0.000     0.000     0.000       131

        accuracy                          0.144      1003
       macro avg      0.012     0.083     0.021      1003
    weighted avg      0.021     0.144     0.036      1003

                  live  safety  price  quiet  dining  nightlife  \
li

In [64]:
cluster_map={0: 'general', 1: 'general', 2: 'general', 3: 'general', 4: 'general', 5: 'transit-location',
6: 'transit-location', 7: 'transit-location', 8: 'green-culture', 9: 'price',
10: 'touristy', 11: 'multicultural', 12: 'price', 13: 'general', 14: 'price', 15: 'shopping', 
16: 'general', 17: 'price', 18: 'general', 19: 'general'}

#print '--- Results on %s domain ---' % (args.domain)
test_labels = r'..\preprocessed_data\sentihood\\test_label.txt'
#test_labels = '../preprocessed_data/sentihood/test_labels_filtered.txt' #changing test_label file to test_labels_filtered : SD
prediction(test_labels, aspect_probs, cluster_map, domain='Sentihood')

NameError: name 'aspect_probs' is not defined

In [15]:
aspect_file='output_dir\\aspect.log10'

file=open(aspect_file)

asp=[line.split('],') for line in file]

In [18]:
len(asp[0])

10

In [25]:
asp_dict=[i.split(': [') for i in asp[0]]

In [28]:
asp_dict[int(asp_dict[0][0].lstrip('{'))]=

0

In [31]:
asp_dict[0][1].strip(")

SyntaxError: EOL while scanning string literal (<ipython-input-31-d494f6437b9d>, line 1)

In [35]:
string = asp_dict[0][1].replace('"','')

In [36]:
string

"'even', 'load', 'cheapest', 'mths', 'posh', 'enough', 'river', 'alternatively', 'lane', 'dump', 'never', 'south', 'min', 'mentioned', 'beautiful', 'case', 'nearer', 'towards', 'going', 'close', 'probably', 'complex', 'first', 'garden', 'rest', 'little', 'located', 'lovely', 'pretty', 'reach', 'recommended', 'way', 'willing', 'called', 'worst', 'also', 'feel', 'want', 'road', 'top', 'near', 'safest', 'perhaps', 'quiet', 'especially', 'north', 'born', 'anywhere', 'really', 'know', 'richmond', 'bet', 'crime', 'considered', 'cheap', 'move', 'could', 'somewhere', 'train', 'bit', 'personally', 'take', 'safer', 'might', 'looking', 'money', 'safe', 'working', 'line', 'affordable', 'pleasant', 'direct', 'say', 'ok', 'still', 'living', 'etc', 'flat', 'think', 'bad', 'right', 'better', 'look', 'dangerous', 'surrey', 'suggest', 'either', 'stay', 'borough', 'cheaper', 'away', 'far', 'recommend', 'go', 'best', 'avoid', 'would', 'try', 'location1', 'location2'"

In [39]:
s='"'even','
s.replace('"','')

SyntaxError: invalid syntax (<ipython-input-39-2938dae57f87>, line 1)